In [1]:
import pandas as pd
import glob

# CONFIGURACIÓN DE COMPORTAMIENTO

In [2]:
# VARIABLES
LIMITE_ESTUDIANTES_CURSO = 550
JORNADAS = ["MAÑANA", "TARDE", "NOCHE"]
# COLUMNAS IMPORTANTES
COLUMNA_JORNADAS = "Seleccione por lo menos dos (2) jornadas en las que tiene disponibilidad de tiempo para presentar la prueba de caracterización o clasificación."
COLUMNA_DOCUMENTO = "Número de Documento"
COLUMNA_NOMBRE = "Nombres"
COLUMNA_APELLIDO = "Apellidos"
COLUMNA_TELEFONO = "Teléfono Celular que usa"
COLUMNA_EMAIL = "Correo electrónico personal que usa"
COLUMNA_CODIGO = "Codigo"

In [3]:
# FUNCIONES
def inicializarCursos(nombre):
    return {
        "NOMBRE_CURSO": nombre,
        "CANTIDAD_ACTUAL": 0,
        "DATOS": []
    }

def inicializarGrupos(grupos, jornadas):
    cont = 1
    for jornada in jornadas:
        for i in range(cont, 10, 3):
            nombre = "CA" + str(i)
            if jornada in grupos:
                grupos[jornada]['CURSOS'].append(inicializarCursos(nombre))
            else:
                grupos[jornada] = {
                    'CURSOS': [inicializarCursos(nombre)],
                    'COSTO': 0
                }
        cont +=1

def menorCostoCurso(grupos, jornada):
    min_cost = LIMITE_ESTUDIANTES_CURSO*2
    nom_curso = ""
    index = -1
    i = 0
    for curso in grupos[jornada]['CURSOS']:
        cantidad = curso['CANTIDAD_ACTUAL']
        if  cantidad < min_cost and cantidad < LIMITE_ESTUDIANTES_CURSO:
            min_cost = cantidad
            nom_curso = curso['NOMBRE_CURSO']
            index = i
        i += 1
    if min_cost > LIMITE_ESTUDIANTES_CURSO:
        raise Exception("Curso por encima del límite")
    return index, nom_curso

def tokenizar(string):
    rta = []
    # 0 => Mañana
    # 1 => Tarde
    # 2 => Noche
    jornadas = string.split(";")
    for jornada in jornadas:
        if (jornada == "Jornada de la mañana"):
            rta.append("MAÑANA")
        elif (jornada == "Jornada de la tarde"):
            rta.append("TARDE")
        elif (jornada == "Jornada de la noche"):
            rta.append("NOCHE")
    return rta

# EJECUCIÓN

In [4]:
archivos = glob.glob('./INPUT/BALANCE/*.xlsx')
dfs = []
for archivo in archivos:
    dfs.append(pd.read_excel(archivo, engine = 'openpyxl'))
datos = pd.concat(dfs, ignore_index = True)
#duplicatos = pd.concat(g for _, g in datos.groupby(COLUMNA_DOCUMENTO) if len(g) > 1)

In [5]:
GRUPOS = {}
inicializarGrupos(GRUPOS, JORNADAS)
for index, row in datos.iterrows():
    jornadas = tokenizar(row[COLUMNA_JORNADAS])    
    min_costo = float("inf")
    nom_jornada = ""
    # Busca el día con menor costo
    for jornada in jornadas:
        if GRUPOS[jornada]['COSTO'] < min_costo:
            min_costo = GRUPOS[jornada]['COSTO']
            nom_jornada = jornada
    # Aumenta el costo
    icurso, curso = menorCostoCurso(GRUPOS, nom_jornada)
    GRUPOS[nom_jornada]['COSTO'] += 1
    GRUPOS[nom_jornada]['CURSOS'][icurso]['CANTIDAD_ACTUAL'] += 1
    GRUPOS[nom_jornada]['CURSOS'][icurso]['DATOS'].append([
        row[COLUMNA_CODIGO],
        row[COLUMNA_DOCUMENTO],
        row[COLUMNA_NOMBRE].upper(),
        row[COLUMNA_APELLIDO].upper(),
        row[COLUMNA_TELEFONO],
        row[COLUMNA_EMAIL],
        nom_jornada,
        curso
    ])

# RESULTADOS

In [6]:
# Creación de los DataFrames
dtfms = []
for jornada in GRUPOS:
    for cursos in GRUPOS[jornada]['CURSOS']:
        df = pd.DataFrame(cursos['DATOS'], columns = [
            "CÓDIGO",
            "DOCUMENTO",
            "NOMBRES",
            "APELLIDOS",
            "TELEFONO",
            "EMAIL",
            "JORNADA",
            "CURSO"
        ])
        dtfms.append(df)
        df.to_excel("./OUTPUT/BALANCE/JORNADA_" + jornada + "_CURSO_" + cursos['NOMBRE_CURSO'] + ".xlsx", index = False)

In [7]:
pd.concat(dtfms, ignore_index = True).to_excel("./OUTPUT/BALANCE/CONSOLIDADO.xlsx", index = False)